In [6]:
import numpy as np

In [7]:
def calculate_material_co2(type, weight):

    materials = {
        "steel": 1.37,
        "stainless_steel": 6.15,
        "stainless steel": 6.15,
        "aluminium": 8.24,
        "aluminum": 8.24,
        "copper": 2.6
    }
    
    if type in materials:
        return materials[type] * weight
    return 0

In [8]:
def calculate_lca(
    training_energy_kwh,
    training_duration_hours,
    carbon_intensity_kgCO2_per_kWh,
    inference_energy_per_query_kwh,
    avg_queries_per_day,
    model_lifetime_years,
):
    """
    Beregner LCA for en AI-modell basert på brukerinput.
    
    :param training_energy_kwh: Strømforbruk (kWh) per time for trening
    :param training_duration_hours: Antall timer modellen trenes
    :param carbon_intensity_kgCO2_per_kWh: CO2-utslipp per kWh strøm
    :param inference_energy_per_query_kwh: Strømforbruk per forespørsel under inferens
    :param avg_queries_per_day: Gjennomsnittlig antall forespørsler per dag
    :param model_lifetime_years: Antall år modellen er i bruk
    :param material_savings_kgCO2: CO2 spart ved materialoptimalisering
    :return: Totalt karbonavtrykk av modellen med materialbesparelser
    """
    
    training_emissions = training_energy_kwh * training_duration_hours * carbon_intensity_kgCO2_per_kWh
    
    inference_emissions = (
        inference_energy_per_query_kwh * avg_queries_per_day * 365 * model_lifetime_years * carbon_intensity_kgCO2_per_kWh
    )
    
    total_emissions = training_emissions + inference_emissions
    
    return {
        "Trening Forbruk": training_emissions,
        "Forespørsel Forbruk": inference_emissions,
        "Totalt Forbruk": total_emissions,
    }


In [9]:
def calculate_mfa(LCA, material_savings_kgCO2):
  """
    Beregner materialbesparelser i kgCO2e basert på LCA og materialbesparelser.
    
    :param LCA: Totalt karbonavtrykk av modellen
    :param material_savings_kgCO2: CO2 spart ved materialoptimalisering
    :return: Totalt karbonavtrykk av modellen med materialbesparelser
  """
    
  total_emissions = LCA["Totalt Forbruk"]
    
  total_emissions -= material_savings_kgCO2
    
  return {
          "Materialbesparelser": material_savings_kgCO2,
          "Netto Karbonavtrykk": total_emissions,
        }

In [20]:
if __name__ == "__main__":

    training_energy_kwh = 500 # Strømforbruk under trening (kWh/time)
    training_duration_hours = 8760 # Total treningsvarighet (timer)
    carbon_intensity_kgCO2_per_kWh = 0.011 # CO2-intensitet i strømnettet (kg CO2/kWh)
    inference_energy_per_query_kwh = 0.0005 # Strømforbruk per forespørsel (kWh)
    avg_queries_per_day = 1000 # Antall forespørsler per dag
    model_lifetime_years = 1 # Forventet levetid for modellen (år)

    steel = calculate_material_co2("steel", 60*1000*model_lifetime_years)
    stainless_steel = calculate_material_co2("stainless steel", 60*100*model_lifetime_years)
    aluminium = calculate_material_co2("aluminium", 60*50*model_lifetime_years)
    copper = calculate_material_co2("copper", 60*100*model_lifetime_years)
    material_savings_kgCO2 = steel + stainless_steel + aluminium + copper # CO2 spart ved materialoptimalisering (kg CO2)

    lca = calculate_lca(
        training_energy_kwh,
        training_duration_hours,
        carbon_intensity_kgCO2_per_kWh,
        inference_energy_per_query_kwh,
        avg_queries_per_day,
        model_lifetime_years
    )

    mfa = calculate_mfa(lca, material_savings_kgCO2)

    print("\nLCA og MFA-beregninger for AI-modellen:")
    print(f"\nType{'':<20}| Mengde CO2 i kg\n---------------------------------------------------")
    for key, value in lca.items():
        print(f"{key:<24}| {value:.2f}")
    for key, value in mfa.items():
        print(f"{key:<24}| {value:.2f}")



LCA og MFA-beregninger for AI-modellen:

Type                    | Mengde CO2 i kg
---------------------------------------------------
Trening Forbruk         | 48180.00
Forespørsel Forbruk     | 6.02
Totalt Forbruk          | 48186.02
Materialbesparelser     | 478260.00
Netto Karbonavtrykk     | -430073.98
